In [123]:
import os
import midas_touch2 as mt2
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import stock_mapping as sm
import warnings
import json
import datetime

pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

# if not mt5.initialize():
#     print("initialize() failed")
# else:
#     print("MT5 successfully initialised.\n")

In [122]:
current_dir = os.getcwd()
#current_path = os.path.abspath(__file__)
#current_dir = os.path.dirname(current_path)
config_file_name = 'config.json'
config_file_path = os.path.join(current_dir, config_file_name)
with open(config_file_path, 'r') as config_file:
        config = json.load(config_file)

symbols = config['symbols']
api_token = config['api_token']
mapping_file = config['mapping_file_path']
sleep_time = config['sleep_time']
account=config['execute_account']
password=config['password']
server=config['server']
Port_TPSL = config['Port_TPSL']
starting_cash = config['starting_cash']

In [94]:
authorized=mt5.login(account, password=password, server=server)
if authorized:
    print(mt5.account_info())
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))


AccountInfo(login=2117, trade_mode=0, leverage=1, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=1, currency_digits=2, fifo_close=False, balance=999962508.32, credit=0.0, profit=0.0, equity=999962508.32, margin=2724.0, margin_free=999959784.32, margin_level=36709343.1835536, margin_so_call=50.0, margin_so_so=30.0, margin_initial=2724.0, margin_maintenance=2724.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Jeff Ng', server='KIBB-Prod', currency='MYR', company='Kenanga Investment Bank Berhad')


In [95]:
mt5_symbol = sm.get_ticker_list(symbols, file_path=mapping_file)
symbols = [symbol + "SE" for symbol in symbols]
price_df = mt2.download_stocks(symbols, "d", api_token, num_days=20)
price_df = pd.DataFrame(price_df)
price_df.columns = mt5_symbol
weights_df, latest_weights = mt2.run_portfolio(price_df, alpha_n=5)
print(f'latest_weights: {latest_weights}')
latest_weights = mt2.generate_random_series(mt5_symbol)
#ADD DUMMY WEIGHTS HERE TO TEST
total_investment = starting_cash
previous_allocation_df = mt2.get_folder(folder_name='daily_allocation')
previous_portfolio_df = mt2.get_folder(folder_name='daily_portfolio')

latest_weights: RHBBANK    0.511603
MAYBANK    0.157264
YNHPROP    0.331132
Name: 2023-12-04, dtype: float64


In [96]:
if previous_allocation_df is not None and previous_portfolio_df is not None:
    prev_holding = previous_allocation_df['Holding Units']
    prev_entry = previous_allocation_df['Entry Units']
    prev_exit = previous_allocation_df['Exit Units']
    total_investment = abs(previous_portfolio_df['Total Value'].iloc[-1])
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    allocation_df['Holding Units'] = prev_holding + prev_entry - prev_exit
else: #first run
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    print("This is the initial run. No previous allocation data available.")
    allocation_df['Holding Units'] = 0

# Calculate Entry and Exit Units
allocation_df['Entry Units'] = allocation_df.apply(
    lambda row: max(row['Allocated Units'] - row['Holding Units'], 0), axis=1)
allocation_df['Exit Units'] = allocation_df.apply(
    lambda row: max(row['Holding Units'] - row['Allocated Units'], 0), axis=1)
numeric_cols = allocation_df.select_dtypes(include=['number']).astype(float)
for col in numeric_cols.columns:
    allocation_df[col] = numeric_cols[col]
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,RHBBANK,5.45,600.0,0.354587,1100.0,0.0,500.0
1,MAYBANK,9.00,400.0,0.416825,0.0,400.0,0.0
2,YNHPROP,4.95,400.0,0.228587,600.0,0.0,200.0


In [97]:
import time
order_ids = mt2.execute_trades_from_data(allocation_df)
time.sleep(5)
orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]
print(f'orders_list: {orders_list}')
deleted_volumes = mt2.delete_orders(orders_list)
print(f'deleted_volumes: {deleted_volumes}')

Placing exit order for RHBBANK
Order executed successfully, transaction ID = 586660
Placing entry order for MAYBANK
Order executed successfully, transaction ID = 586661
Placing exit order for YNHPROP
Order executed successfully, transaction ID = 586662
Order with ticket 586660 not found or error occurred, error code = (1, 'Success')
Order with ticket 586661 not found or error occurred, error code = (1, 'Success')
Order with ticket 586662 not found or error occurred, error code = (1, 'Success')
orders_list: [None, None, None]
deleted_volumes: []


In [98]:
def get_opening_orders(stock_symbols, weights):
    if not mt5.initialize():
        print("initialize() failed, error code =", mt5.last_error())
        return None, pd.DataFrame()  # Return an empty DataFrame if MT5 fails to initialize

    all_positions = []
    open_prices = {}
    for symbol in stock_symbols:
        positions = mt5.positions_get(symbol=symbol)
        today_open_price = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_D1, 0, 1)
        if positions is None:
            print(f"No positions for {symbol}, error code={mt5.last_error()}")
        elif len(positions) > 0:
            all_positions.extend(list(positions))
            open_prices[symbol] = today_open_price[0]['open']
    df = pd.DataFrame()
    if all_positions:
        df = pd.DataFrame(all_positions, columns=all_positions[0]._asdict().keys())
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df = df.drop(['time_msc', 'time_update', 'time_update_msc', 'identifier', 'reason', 
                      'sl', 'tp', 'swap', 'comment', 'external_id'], axis=1)
        df['position_change'] = (df['price_current'] / df['price_open']) - 1
        df['value'] = df['volume'] * df['price_open']
        df['weights'] = df['symbol'].map(weights)
        df['portfolio_change_position'] = df['weights'] * df['position_change']
    total_portfolio_change_position = df['portfolio_change_position'].sum() if not df.empty else 0
    print(f"Total portfolio change position: {total_portfolio_change_position:.4%}")
    return total_portfolio_change_position, df
mt5_symbol = ['RHBBANK','MAYBANK','YNHPROP']
port_change, opening_df = get_opening_orders(mt5_symbol, latest_weights)
opening_df

Total portfolio change position: 0.0000%


,ticket,time,type,magic,volume,price_open,price_current,profit,symbol,position_change,value,weights,portfolio_change_position
0,586657,2023-12-04 21:03:46,0,1116,600.0,5.45,5.45,0.0,RHBBANK,0.0,3270.0,0.354587,0.0
1,586661,2023-12-04 21:06:58,0,1116,400.0,9.00,9.00,0.0,MAYBANK,0.0,3600.0,0.416825,0.0
2,586659,2023-12-04 21:05:24,0,1116,400.0,4.95,4.95,0.0,YNHPROP,0.0,1980.0,0.228587,0.0


In [140]:

def get_filled_orders(symbol_list):
    from_date = datetime.datetime.now()
    to_date = datetime.datetime.now() + datetime.timedelta(days=1)

    print(f"from_date: {from_date}")
    print(f"to_date: {to_date}")
    all_deals = []
    for symbol in symbol_list:
        deals = mt5.history_orders_get(from_date, to_date)
        if deals is None:
            print(f"No deals for {symbol}, error code={mt5.last_error()}")
        elif len(deals) > 0:
            all_deals.extend(list(deals))
    if not all_deals:
        print("No deals found for any of the specified symbols.")
        return pd.DataFrame()
    df = pd.DataFrame(all_deals, columns=all_deals[0]._asdict().keys())
    df['time_done'] = pd.to_datetime(df['time_done'], unit='s')
    # Drop columns
    df = df.drop(['time_setup', 'time_done_msc', 'time_setup_msc', 'time_expiration', 'sl',
                  'type_time', 'type_filling', 'volume_current', 'reason', 'price_open', 'tp',
                  'position_id', 'position_by_id', 'external_id', 'price_stoplimit'], axis=1)
    df = df.drop_duplicates()
    filled_orders = df[df['state'] == 4]
    filled_orders = filled_orders[filled_orders['symbol'].isin(symbol_list)]
    return filled_orders

filled_orders = get_filled_orders(mt5_symbol)
filled_orders
################
current_time = datetime.datetime.now()
# Filter for orders done in the last 5 minutes
five_minutes_ago = current_time - datetime.timedelta(minutes=1)
recent_filled_orders = filled_orders[pd.to_datetime(filled_orders['time_done']) >= five_minutes_ago]
recent_filled_orders

from_date: 2023-12-04 21:59:16.093064
to_date: 2023-12-05 21:59:16.093064


,ticket,time_done,type,state,magic,volume_initial,price_current,symbol,comment
313,586700,2023-12-04 21:58:31,2,4,1116,500.0,5.45,RHBBANK,kkyao2 Entry
314,586701,2023-12-04 21:58:31,3,4,1116,200.0,9.00,MAYBANK,kkyao2 Exit
315,586702,2023-12-04 21:58:31,3,4,1116,300.0,4.95,YNHPROP,kkyao2 Exit


In [141]:
buy_order = recent_filled_orders[recent_filled_orders['type'] == 2]
purchase_cost = (buy_order['price_current'] * buy_order['volume_initial']).sum()
print(f'purchase_cost: {purchase_cost}')
sell_order = recent_filled_orders[recent_filled_orders['type'] == 3]
sell_value = (sell_order['price_current'] * sell_order['volume_initial']).sum()
print(f'sell_value: {sell_value}')
current_cap = 995 -purchase_cost+sell_value
print(f'current_cap: {current_cap}')

purchase_cost: 2725.0
sell_value: 3285.0
current_cap: 1555.0


In [100]:
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,RHBBANK,5.45,600.0,0.354587,1100.0,0.0,500.0
1,MAYBANK,9.00,400.0,0.416825,0.0,400.0,0.0
2,YNHPROP,4.95,400.0,0.228587,600.0,0.0,200.0


In [101]:
# def close_all_positions(df, comment):
#     for _, row in df.iterrows():
#         symbol = row['symbol']
#         volume = row['volume']
#         # Fetch the current bid price for the symbol
#         symbol_info = mt5.symbol_info_tick(symbol)
#         if symbol_info is None:
#             print(f"Failed to get symbol info for {symbol}, error code =", mt5.last_error())
#             continue
#         current_bid = symbol_info.bid
#         request = {
#             "action": mt5.TRADE_ACTION_DEAL,
#             "symbol": symbol,
#             "volume": volume,
#             "type": mt5.ORDER_TYPE_SELL,
#             "price": current_bid,
#             "deviation": 20,  # Deviation in points
#             "magic": 20231207,       # Magic number, if needed
#             "comment": comment,
#             "type_time": mt5.ORDER_TIME_GTC,
#             "type_filling": mt5.ORDER_FILLING_RETURN,
#         }
#         result = mt5.order_send(request)
#         if result.retcode != mt5.TRADE_RETCODE_DONE:
#             print(f"Failed to close position for {symbol}, error code: {result.retcode}")
#         else:
#             print(f"Position on {symbol} closed successfully.")

# close_all_positions(opening_df, "Closing all positions")

In [102]:
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,RHBBANK,5.45,600.0,0.354587,1100.0,0.0,500.0
1,MAYBANK,9.00,400.0,0.416825,0.0,400.0,0.0
2,YNHPROP,4.95,400.0,0.228587,600.0,0.0,200.0


In [103]:
if not opening_df.empty:
    print("Opening orders:")
else:
    print("No opening orders found.")
opening_df

Opening orders:


,ticket,time,type,magic,volume,price_open,price_current,profit,symbol,position_change,value,weights,portfolio_change_position
0,586657,2023-12-04 21:03:46,0,1116,600.0,5.45,5.45,0.0,RHBBANK,0.0,3270.0,0.354587,0.0
1,586661,2023-12-04 21:06:58,0,1116,400.0,9.00,9.00,0.0,MAYBANK,0.0,3600.0,0.416825,0.0
2,586659,2023-12-04 21:05:24,0,1116,400.0,4.95,4.95,0.0,YNHPROP,0.0,1980.0,0.228587,0.0


purchase_cost: 4680.0
sell_value: 4905.0
current_cap: 995.0


In [108]:
def create_daily_log(allocation_df, opening_df, starting_cash=10000, log_folder="daily_portfolio", log_file_name="balance.csv"):
    log_file_path = os.path.join(log_folder, log_file_name)
    if os.path.exists(log_folder) and os.path.isfile(log_file_path):
        previous_log = pd.read_csv(log_file_path)
        initial_capital = previous_log.iloc[-1]['Cash Balance']
    else:
        initial_capital = starting_cash
    cash_balance = initial_capital
    # Calculate purchase cost
    buy_order = recent_filled_orders[recent_filled_orders['type'] == 2]
    purchase_cost = (buy_order['price_current'] * buy_order['volume_initial']).sum()
    cash_balance -= purchase_cost
    # Calculate portfolio value
    portfolio_value = (opening_df['price_current'] * opening_df['volume']).sum()
    # Calculate sales proceeds
    sell_order = recent_filled_orders[recent_filled_orders['type'] == 3]
    sales_proceed = (sell_order['price_current'] * sell_order['volume_initial']).sum()
    cash_balance += sales_proceed
    # Total value
    total_value = cash_balance + portfolio_value
    # Create the log dataframe
    log_df = pd.DataFrame({
        'Date': [pd.to_datetime('today').date()],
        'Initial Capital': [initial_capital],
        'Cash Balance': [cash_balance],
        'Portfolio Value': [portfolio_value],
        'Purchase Cost': [purchase_cost],
        'Sales Proceeds': [sales_proceed],
        'Total Value': [total_value]
    })
    return log_df
log_df = create_daily_log(allocation_df, opening_df)

mt2.save_df_to_csv(log_df, 
                   folder_name='daily_portfolio', 
                   file_name='balance',
                   append=True)


Data appended to daily_portfolio\balance.csv


In [112]:
log_df

,Date,Initial Capital,Cash Balance,Portfolio Value,Purchase Cost,Sales Proceeds,Total Value
0,2023-12-04,1035.0,1150.0,8850.0,3600.0,3715.0,10000.0


In [110]:
mt2.save_df_to_csv(allocation_df,
                     folder_name='daily_allocation',
                     file_name='allocation')

Data saved in daily_allocation\allocation_2023-12-04.csv
